# Multimodal Cancer Detection using the Intel® Transfer Learning Tool API

This application is a multimodal solution for predicting cancer diagnosis using categorized contrast enhanced mammography data and radiology notes. It trains two models - one for image classification and the other for text classification - which can be combined into an ensemble classifier.

## Import Dependencies and Setup Directories

In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import torch

from transformers import EvalPrediction, TrainingArguments

# tlt imports
from tlt.datasets import dataset_factory
from tlt.models import model_factory

# Specify the root directory where the images and annotations are located
dataset_dir = os.path.join(os.environ["DATASET_DIR"]) if "DATASET_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "dataset")

# Specify a directory for output
output_dir = os.environ["OUTPUT_DIR"] if "OUTPUT_DIR" in os.environ else \
    os.path.join(os.environ["HOME"], "output")

print("Dataset directory:", dataset_dir)
print("Output directory:", output_dir)

## Dataset

Download the images and radiology annotations from https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=109379611

Image files should have the .jpg extension and be arranged in subfolders for each class. The annotation file should be a .csv. The data directory should look something like this:

```
brca
  ├── annotation
  │   └── annotation.csv
  └── vision_images
      ├── Benign
      │   ├── P100_L_CM_CC.jpg
      │   ├── P100_L_CM_MLO.jpg
      │   └── ...
      ├── Malignant
      │   ├── P102_R_CM_CC.jpg
      │   ├── P102_R_CM_MLO.jpg
      │   └── ...
      └── Normal
          ├── P100_R_CM_CC.jpg
          ├── P100_R_CM_MLO.jpg
          └── ...
```

In [ ]:
# User input needed - supply the path to the images in the dataset_dir according to your system
source_image_path = os.path.join(dataset_dir, 'brca', 'vision_images')
image_path = source_image_path

# User input needed - supply the path and name of the annotation file in the dataset_dir
source_annotation_path = os.path.join(dataset_dir, 'brca', 'annotation', 'annotation.csv')
annotation_path = source_annotation_path
label_col = 3  # Index of the label column in the data file

### Optional: Group Data by Patient ID

This section is not required to run the workload, but it is helpful to assign all of a subject's records to be entirely in the train set or test set. This section will do a random stratification based on patient ID and save new copies of the grouped data files.

In [ ]:
from data_utils import split_images, split_annotation

grouped_image_path = '{}_grouped'.format(source_image_path)

if os.path.isdir(grouped_image_path):
    print("Grouped directory already exists and will be used: {}".format(grouped_image_path))
else:
    split_images(source_image_path, grouped_image_path)

image_path = grouped_image_path

In [ ]:
file_dir, file_name = os.path.split(source_annotation_path)
grouped_annotation_path = os.path.join(file_dir, '{}_grouped.csv'.format(os.path.splitext(file_name)[0]))

if os.path.isfile(grouped_annotation_path):
    print("Grouped annotation already exists and will be used: {}".format(grouped_annotation_path))
else:
    train_dataset = split_annotation(file_dir, file_name, image_path)
    train_dataset.to_csv(grouped_annotation_path)
    print('Grouped annotation saved to: {}'.format(grouped_annotation_path))

annotation_path = grouped_annotation_path
label_col = 1  # Index of the label column in the grouped data file

## Model 1: Image Classification with TensorFlow

### Get the Model and Dataset
Call the model factory to get a pretrained model from TensorFlow Hub and the dataset factory to load the images from their location. The `get_model` function returns a model object that will later be used for training. We will use resnet_v1_50 by default.

In [ ]:
model = model_factory.get_model(model_name="resnet_v1_50", framework='tensorflow')

# Load the dataset from the custom dataset path
dataset = dataset_factory.load_dataset(dataset_dir=image_path,
                                       use_case='image_classification',
                                       framework='tensorflow')

print("Class names:", str(dataset.class_names))

### Data Preparation
Once you have your dataset loaded, use the following cell to preprocess the dataset. We split the images into training and validation subsets, resize them to match the model, and then batch the images.

In [ ]:
batch_size = 16
if 'grouped' not in image_path:
    # Split if not pre-defined
    dataset.shuffle_split(train_pct=.80, val_pct=0.0, test_pct=0.2)
dataset.preprocess(model.image_size, batch_size=batch_size)

### Transfer Learning

This step calls the model's train function with the dataset that was just prepared. The training function will get the TFHub feature vector and add on a dense layer based on the number of classes in the dataset. The model is then compiled and trained based on the number of epochs specified in the argument. We also add two more dense layers using the `extra_layers` parameter.

To optionally insert additional dense layers between the base model and output layer, `extra_layers=[1024, 512]` will insert two dense layers, the first with 1024 neurons and the second with 512 neurons.

In [ ]:
history = model.train(dataset, output_dir=output_dir, epochs=5, seed=10, extra_layers=[1024, 512], do_eval=False)

In [ ]:
metrics = model.evaluate(dataset, use_test_set=True)
for metric_name, metric_value in zip(model._model.metrics_names, metrics):
    print("{}: {}".format(metric_name, metric_value))

### Save the Computer Vision Model

In [ ]:
saved_model_dir = model.export(output_dir)

## Model 2: Text Classification with PyTorch

### Get the Model and Dataset
Now we will call the model factory to get a pretrained model from Hugging Face and load the annotation file using the dataset factory. We will use clinical-bert for this part.

In [ ]:
# Set up NLP parameters
model_name = 'clinical-bert'
seq_length = 64
batch_size = 5
quantization_criterion = 0.05
quantization_max_trial = 50
epochs = 3

In [ ]:
model = model_factory.get_model(model_name=model_name, framework='pytorch')

In [ ]:
# Create a label map function and reverse label map for the dataset
def label_map_func(label):
        if label == 'Benign':
            return 0
        elif label == 'Malignant':
            return 1
        elif label == 'Normal':
            return 2
    
reverse_label_map = {0: 'Benign', 1: 'Malignant', 2: 'Normal'}

In [ ]:
file_dir, file_name = os.path.split(annotation_path)
dataset = dataset_factory.load_dataset(dataset_dir=file_dir,
                       use_case='text_classification',
                       framework='pytorch',
                       dataset_name='brca',
                       csv_file_name=file_name,
                       label_map_func=label_map_func,
                       class_names=['Benign', 'Malignant', 'Normal'],
                       header=True,
                       label_col=label_col,
                       shuffle_files=True)

### Data Preparation

In [ ]:
dataset.preprocess(model.hub_name, batch_size=batch_size, max_length=seq_length)
dataset.shuffle_split(train_pct=0.67, val_pct=0.33)

### Transfer Learning

This step calls the model's train function with the dataset that was just prepared. The training function will get the pretrained model from HuggingFace and add on a dense layer based on the number of classes in the dataset. The model is then trained using an instance of Hugging Face Trainer for the number of epochs specified. If desired, a native PyTorch loop can be invoked instead of Trainer by setting `use_trainer=False`.

In [ ]:
history = model.train(dataset, output_dir, epochs=epochs, use_trainer=True)

### Save the NLP Model

In [ ]:
model.export(output_dir)

### Int8 Quantization

We can use the [Intel® Extension for Transformers](https://github.com/intel/intel-extension-for-transformers) to quantize the trained model for faster inference. If you want to run this part of the notebook, make sure you have `intel-extension-for-transformers` installed in your environment.

In [ ]:
!pip install intel-extension-for-transformers==1.0.1

In [ ]:
from intel_extension_for_transformers.optimization.trainer import NLPTrainer
from intel_extension_for_transformers.optimization import metrics, objectives, OptimizedModel, QuantizationConfig

In [ ]:
# Set up quantization config
tune_metric = metrics.Metric(
    name="eval_accuracy",
    greater_is_better=True,
    is_relative=True,
    criterion=quantization_criterion,
    weight_ratio=None,
)

objective = objectives.Objective(
    name="performance", greater_is_better=True, weight_ratio=None
)

quantization_config = QuantizationConfig(
    approach="PostTrainingDynamic",
    max_trials=quantization_max_trial,
    metrics=[tune_metric],
    objectives=[objective],
)

# Set up metrics computation
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [ ]:
quantizer = NLPTrainer(model=model._model,
                       train_dataset=dataset.train_subset,
                       eval_dataset=dataset.validation_subset,
                       compute_metrics=compute_metrics,
                       tokenizer=dataset._tokenizer)
quantized_model = quantizer.quantize(quant_config=quantization_config)

In [ ]:
results = quantizer.evaluate()
eval_acc = results.get("eval_accuracy")
print("Final Eval Accuracy: {:.5f}".format(eval_acc))

## Citations

### Data Citation
Khaled R., Helal M., Alfarghaly O., Mokhtar O., Elkorany A., El Kassas H., Fahmy A. <b>Categorized Digital Database for Low energy and Subtracted Contrast Enhanced Spectral Mammography images [Dataset].</b> (2021) The Cancer Imaging Archive. DOI:  [10.7937/29kw-ae92](https://doi.org/10.7937/29kw-ae92)

### Publication Citation
Khaled, R., Helal, M., Alfarghaly, O., Mokhtar, O., Elkorany, A., El Kassas, H., & Fahmy, A. <b>Categorized contrast enhanced mammography dataset for diagnostic and artificial intelligence research.</b> (2022) Scientific Data, Volume 9, Issue 1. DOI: [10.1038/s41597-022-01238-0](https://doi.org/10.1038/s41597-022-01238-0)

### TCIA Citation
Clark K, Vendt B, Smith K, Freymann J, Kirby J, Koppel P, Moore S, Phillips S, Maffitt D, Pringle M, Tarbox L, Prior F. <b>The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository</b>, Journal of Digital Imaging, Volume 26, Number 6, December, 2013, pp 1045-1057. DOI: [10.1007/s10278-013-9622-7](https://doi.org/10.1007/s10278-013-9622-7)